In [33]:
import pandas as pd
import numpy as np
import csv
import pymysql as sql

In [34]:
#connect to company DB
conn = sql.connect(host='wingz-platform-read001.c8voyumknq5z.us-west-1.rds.amazonaws.com',
                            user='wingz-read-only', password='4P4v53S256hW7Z2X',
                            db='wingz-prod')

In [35]:
#function to query results into a pandas dataframe
def query(query):
    return pd.read_sql(query, con=conn)

In [36]:
ridersQuery = """select rider_id, COUNT(rider_id) as 'Rides Booked', 
sum(case when charged_to_rider = 0 then 0 else 1 end) as '# of rides paid',
SUM(charged_to_rider) as 'Total Cost', 
airport from rides where charged_to_rider is not null group by rider_id
"""

ridersDF = query(ridersQuery)


In [38]:
driversQuery = """
SELECT sub.nom, sub.prenom, rides.driver_id, count(rides.driver_id) AS "# of rides accepted", 
sum(case when unavailable_requested_driver_fallback = 0 then 1 else 0 end) as "# of rides provided"
FROM rides, (select utilisateurs.id_utilisateur,utilisateurs.prenom,utilisateurs.nom
from utilisateurs
where utilisateurs.driver_activated_at is not NULL and utilisateurs.deleted_at is NULL and utilisateurs.id_driver_info is not NULL) sub
WHERE parent_id is NOT null AND state_id in (6,7,8) AND date_reservation > '2016-01-01'
and rides.driver_id = sub.id_utilisateur
group by driver_id
"""


,nom,prenom,driver_id,# of rides accepted,# of rides provided
0,Romand,Jeremie,19810,8,3.0
1,Taguchi,Dennis,31267,150,109.0
2,Baumbach,Christof,31294,1,0.0
3,Shiver,Keith,33738,22,15.0
4,Vasquez Gonzalez,Miguelangel,40795,106,101.0
5,Escobar,Alex,43159,46,38.0
6,Vinnicov,Yuri,43330,30,29.0
7,Bae,Nelson,43633,115,96.0
8,Shakya,Yugesh,43926,131,122.0
9,Piperdi,Yusuf,44363,311,281.0


In [39]:
activatedDriversQuery = """
select utilisateurs.id_utilisateur,utilisateurs.prenom,utilisateurs.nom
from utilisateurs
where utilisateurs.driver_activated_at is not NULL and utilisateurs.deleted_at is NULL and utilisateurs.id_driver_info is not NULL
"""

activatedDriversDF = query(activatedDriversQuery)

,id_utilisateur,prenom,nom
0,15045,Geoff,Mathieux
1,19810,Jeremie,Romand
2,28853,Fred,Gomez
3,31267,Dennis,Taguchi
4,31294,Christof,Baumbach
5,33738,Keith,Shiver
6,40795,Miguelangel,Vasquez Gonzalez
7,43159,Alex,Escobar
8,43330,Yuri,Vinnicov
9,43633,Nelson,Bae
